In [ ]:
## Note to self: t is the sensitive feature

## combining t so it's part of x with the features

In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import seaborn as sns
import os
import random
import datetime

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder 
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import (
    Input, Dense, Conv2D, Flatten, 
    MaxPooling2D, BatchNormalization, Dropout
)

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateScheduler
)
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import Constraint
from tensorflow.keras.optimizers import Adam
from keras.initializers import Constant

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    roc_auc_score,
    accuracy_score
)

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D
from tqdm import tqdm
from keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import (
    Input, Dense, Conv2D, Flatten, 
    MaxPooling2D, BatchNormalization, Dropout
)

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateScheduler
)

from tensorflow.keras.regularizers import Regularizer

from tensorflow.keras.models import Sequential, Model
from keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler

import tarfile

2025-01-28 17:48:25.671074: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-28 17:48:25.671116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-28 17:48:25.672680: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 17:48:25.681297: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def load_compas_data_pcategorizing(data_path, seed_num=0):
    df = pd.read_csv(data_path) 
    sensitive_feature = df["Ethnic_Code_Text"].to_numpy()
    sensitive_feature_label = df["Ethnic_Code_Raw"].to_numpy()
    labels = df["ScoreText_"].to_numpy()
    features = df.drop(["ScoreText_", "Ethnic_Code_Raw", "Ethnic_Code_Text"], axis=1).to_numpy()

    #label shape changed  -- ADDED THIS 
    labels = labels.reshape(-1, 1)
    return features, labels, sensitive_feature, sensitive_feature_label

In [ ]:
### ADD IN REGULARIZERS 
import tensorflow.keras.backend as K

def independence_loss(y_true, y_pred):
    """
    Custom loss function to ensure independence between latent representation and sensitive attribute.
    - y_pred: Output of the softmax layer (target cluster assignment)
    - y_true: Sensitive attribute
    """
    # getting the join probability between predicted labels and proxy variable
    joint_distribution = K.mean(y_pred[..., None] * y_true[:, None, :], axis=0)  
    # this is to get the marignal distribution of the predicted labels
    marginal_pred = K.mean(y_pred, axis=0, keepdims=True) 
    # this is to get the marignal distirbution of true labels 
    marginal_true = K.mean(y_true, axis=0, keepdims=True)  ### y_true would be s (the proxy)?

    # this is using KL Divergence -- if P(Y,S) is close to P(Y)P(S), KL divergence is small making them close to dependent
    # note: if Y and S are independent, joint distribution should be P(Y,S) = P(Y)P(S)
    # but since not always possible, KL divergence is used
    independence_penalty = K.sum(joint_distribution * K.log(joint_distribution / (marginal_pred * marginal_true + 1e-8) + 1e-8))

    return independence_penalty

def custom_loss(y_true, y_pred):
    """
    Combines categorical crossentropy with independence loss.
    """
    base_loss = K.categorical_crossentropy(y_true, y_pred)  # Original classification loss
    independence_penalty = independence_loss(y_true, y_pred)  # Additional regularization for independence
    lambda_reg = 0.1  # Regularization weight for independence loss
    return base_loss + lambda_reg * independence_penalty



In [3]:
class ClipConstraint(Constraint):
    """
    Clips model weights to a given range [min_value, max_value].
    Normalizes weights along a specific axis to exurethey sum to1
    """
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, weights):
        w = tf.clip_by_value(weights, self.min_value, self.max_value)
        return w / tf.reduce_sum(w, axis=1, keepdims=True)
    def get_config(self):
        return {'min_value': self.min_value, 'max_value': self.max_value}
        

class VarianceRegularizer(Regularizer):
    """
    Custom regularizer for maximum weight variance.
    Purpose: encourage uniformity among weights -- improve generalization or stability
    """
    
    def __init__(self, factor=0.01):
        self.factor = factor
    
    def __call__(self, x):
        variances = tf.math.reduce_variance(x, axis=1)
        max_variance = tf.reduce_max(variances)
        return self.factor * max_variance
    
    def get_config(self):
        return {'factor': self.factor}


def lr_schedule(epoch):
    """Defines the learning rate schedule."""
    if epoch < 20:
        return 1e-4
    else:
        return 1e-5

def adversarial_loss(y_true, y_pred):
    return -tf.keras.losses.categorical_crossentropy(y_true, y_pred)  # Reverse gradient
        

# X = features 
# s = proxy
# t = sensitive 
# want s_f and t to be independent of each other
def get_model_z(X, s, t, n_z,model_name,epochs=20,verbose=1,var_reg=0.0):
    """
    Defines and trains a clustering model. 
    """
    # Ensuring that d and s are independent of each other 
    
    #Shuffle X, s, and t together
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    
    lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)
    
    X = X[indices]
    s = s[indices]
    t = t[indices]


    input_layer = Input(shape=(X.shape[1],))

    hidden = Dense(1024, activation='relu')(input_layer)
    hidden = Dropout(0.5)(hidden)
    hidden = Dense(128, activation='relu')(hidden)
    hidden = Dropout(0.5)(hidden)
    hidden = Dense(64, activation='relu')(hidden)

    # fair label prediction
    s_f_layer = Dense(s.shape[1], activation='softmax', name='s_f_layer')(hidden)

    # adding adverersarial portion -- predict t from s_f
    adv_hidden = Dense(64, activation='relu')(s_f_layer)
    t_pred_layer = Dense(t.shape[1], activation='softmax', name='t_pred_layer')(adv_hidden)

    model = Model(inputs=input_layer, outputs=[s_f_layer, t_pred_layer])

    model.compile(
        optimizer = Adam(learning_rate=1e-3), 
        loss={'s_f_layer':'categorical_crossentropy', 't_pred_layer':adversarial_loss}, 
        loss_weights={'s_f_layer':1.0, 't_pred_layer':0.1}, # balance learning 
        metrics={'s_f_layer':'accuracy', 't_pred_layer':'accuracy'}
    )

    model_checkpoint_path = os.path.join(repo_root, model_name)
    model_checkpoint_callback = ModelCheckpoint(
            filepath=model_checkpoint_path,
            save_best_only=True,
            monitor='val_loss',
            mode='min',
            verbose=verbose
        )

    model.fit(
        X, 
        {'s_f_layer':s, 't_pred_layer':t}, #multiple output training 
        batch_size=1024, 
        epochs=epochs, 
        validation_split=0.1, 
        callbacks=[model_checkpoint_callback, lr_scheduler]   
    )
    
    best_model = load_model(
        model_checkpoint_path, 
        custom_objects={'ClipConstraint': ClipConstraint,'VarianceRegularizer': VarianceRegularizer, 'adversarial_loss':adversarial_loss}
    )
    return best_model

def pzx(X,best_model,arg_max=True):
    """
    Predict cluster assignments
    """
    # extracting output layer for fair label predictions
    softmax_output_model = Model(inputs=best_model.input, outputs=best_model.get_layer('s_f_layer').output)
    p = softmax_output_model.predict(X)
    if(arg_max):
        p = np.argmax(p,axis=1)
    return p

def set_seed(seed_num): 
    # Set random seed for reproducibility
    seed = seed_num
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [6]:

repo_root = os.path.abspath(os.path.join(os.getcwd(), "..")) # specific for Jupyter Notebook

def run_compas(data_path, output_csv_name, output_model_results, num_clusters=2, num_epochs=40, model_path='best_census_model_inc.h5', num_var_reg=0, seed_num=0):
    
    set_seed(seed_num)

    # labels is the proxy, ethn_from_df is the sensitive feature
    features, labels, sensitive_feature, sensitive_feature_label = load_compas_data_pcategorizing(data_path)
    # labels are already binarized
    best_model = get_model_z(features, labels, sensitive_feature, num_clusters, model_path, num_epochs, num_var_reg)

    p1_tr = pzx(features, best_model, arg_max=True)

    cluster_eg_df = pd.DataFrame({
        "cluster":p1_tr,
        "ethinic_group_numeric":sensitive_feature,
        "ethnic_group_label":sensitive_feature_label, # this is t
        })

    # saving df to csv
    output_folder = os.path.join(repo_root, "compass_category_results")
    os.makedirs(output_folder, exist_ok=True)
    output_csv_path = os.path.join(output_folder, output_csv_name)
    cluster_eg_df.to_csv(output_csv_path, index=False)

    # Group by 'ethinic group' and 'cluster', then count the occurrences of each cluster for each ethnic grouup
    cluster_counts = cluster_eg_df.groupby(["ethnic_group_label", "cluster"]).size().reset_index(name='count')
    
    # total count for each ethinic group 
    eg_totals = cluster_eg_df.groupby("ethnic_group_label")["cluster"].count().reset_index(name='total')
    
    # Merge the counts with the total counts per ethinic group
    merged = pd.merge(cluster_counts, eg_totals, on="ethnic_group_label")
    
    # proportion for each cluster within each ethnic group
    merged['proportion'] = merged['count'] / merged['total']
    result = merged.loc[merged.groupby("ethnic_group_label")["proportion"].idxmax()]
    cluster_groups = result.groupby("cluster")["ethnic_group_label"].apply(list)

    output_model_folder = os.path.join(repo_root, "compass_category_results")
    os.makedirs(output_model_folder, exist_ok=True)
    output_txt_path = os.path.join(output_model_folder, output_model_results)
    with open(output_txt_path, "w") as file:
        file.write("Ethnic Group - Cluster Mapping Based on Highest Proportion:\n")
        file.write("=" * 50 + "\n\n")
        for cluster, eg in cluster_groups.items():
            eg_list = ", ".join(eg)
            file.write(f"Cluster {cluster}: = [{eg_list}]\n")

    #  will need to add evaluate fairness measures at some point
    # evaluate_fairness(p1_tr, sensitive_feature_label)


In [ ]:
## Evaluate whether truly s_f and t are truly independet (true label and sensitive feature)

import seaborn as sns 
import matplotlib.pyplot as plt 
from scipy.stats import chi2_contingency
from sklearn.metric import mutual_info_score

# def evaluate_fairness COME BACK HERE AND FINISH 

In [8]:
parameters = {
    "data_path":"unit_test_data/mock_compass_1000.csv", 
    "output_csv_name": "compas_60epoch.csv", 
    "output_model_results": "compass_40epochs_2.txt", 
    "num_clusters":3, 
    "num_epochs":60, 
    "model_path":'best_compas_model_inc.h5', 
    "num_var_reg":0, 
    "seed_num":0
}

run_compas(**parameters)


Epoch 1: LearningRateScheduler setting learning rate to 0.0001.
Epoch 1/60


/opt/conda/lib/python3.11/site-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 2s 2s/step - loss: 6.3181e-08 - accuracy: 0.5089 - val_loss: 4.8876e-08 - val_accuracy: 0.4100 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001.
Epoch 2/60


/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 1s 639ms/step - loss: 6.3181e-08 - accuracy: 0.5300 - val_loss: 4.8876e-08 - val_accuracy: 0.4100 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001.
Epoch 3/60
1/1 [==============================] - 1s 601ms/step - loss: 6.3181e-08 - accuracy: 0.5300 - val_loss: 4.8876e-08 - val_accuracy: 0.4100 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001.
Epoch 4/60
1/1 [==============================] - 1s 696ms/step - loss: 6.3181e-08 - accuracy: 0.5300 - val_loss: 4.8876e-08 - val_accuracy: 0.4100 - lr: 1.0000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0001.
Epoch 5/60
1/1 [==============================] - 1s 696ms/step - loss: 6.3181e-08 - accuracy: 0.5300 - val_loss: 4.8876e-08 - val_accuracy: 0.4100 - lr: 1.0000e-04

Epoch 6: LearningRateScheduler setting learning rate to 0.0001.
Epoch 6/60
1/1 [==============================] - 1s 886ms/step - loss: 6.3181e-08 - accura

In [13]:
features, labels, ethn_from_df = load_compas_data_pcategorizing(parameters["data_path"], seed_num=0)

In [14]:
features

array([[0, 2, 6, ..., 3, 2, 0],
       [0, 7, 0, ..., 5, 1, 1],
       [0, 5, 0, ..., 3, 1, 1],
       ...,
       [0, 4, 0, ..., 0, 2, 1],
       [0, 2, 3, ..., 1, 4, 0],
       [0, 4, 2, ..., 2, 4, 1]])

In [15]:
labels

array([0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,